# Rayleigh output -> VAPoR input Converter

## Setup

You must set these variables for this notebook to run properly.

- **vapor_version**: Version of Vapor to export for. Must be 2 or 3.
- **vbin**: Locations of `bin` directories for vapor 2 and vapor 3.
- **run_dir**: Directory the simulation was run in. Must end in '/'.
- **data_dir**: Location of Rayleigh's 3D output. Should usually be 'Spherical_3D/'
- **grid_file**: The grid to interpolate from. Usually in the data directory. All interpolated variables must use the same grid.
- **vapor_file**: Name of vapor output data file. Omit the file type (it will be added automatically).
- **cube_size_n**: Size of cartesian grid to interpolate onto. Give the number of grid points on a side.
- **rayleigh_bin**: Location of the `bin` subdirectory in the Rayleigh repo. `interp3d` must be in this directory.
- **temp_dir**: Optional location for temporary files. The directory must already exist. It is not created automatically.
- **diter**: The number of iterations between files to convert. Does not have to match the original output frequency.
- **first/last_iter**: The range of iterations to convert.
- **overwrite**: Whether it should overwrite files from previous runs.

In [ ]:
vapor_version=3
if (vapor_version == 3):
    vbin='/custom/software/VAPOR3-3.2.0-Linux/bin'
    # FOR MAC, it will be something like: vroot=`/Applications/vapor.app/Contents/MacOS`
else:
    vbin='/custom/software/vapor-2.6.0.RC0/bin'

run_dir = "./"
data_dir = 'Spherical_3D/'
    
grid_file = run_dir + data_dir + 'GRID_FILE_HERE'

vapor_file = run_dir+'vapor_input'

cube_size_N=256

rayleigh_root='/YOUR/RAYLEIGH/FORK/HERE/bin'

tempdir='.'

diter = 10
first_iter = 0
last_iter  = 10

overwrite = True

## Scalar Quantities

This defines the scalars quantities to convert.

- **Key**: name of the variable as it should appear in VAPoR
- **code**: quantity code used by Rayleigh
- **rm_spherical_mean**: whether to subtract the mean from the data. Useful when the deviation is small relative to the mean. Defaults to `False`.
- **radius_limits**: (lower bound, upper bound). Points outside these bounds will be set to zero. Defaults to `(None, None)`.

In [ ]:
scalars = {
    "temp": {
        "code": 501,
        "rm_spherical_mean": True,
        "radius_limits": (None, None)
    },
    "vphi": {
        "code": 3,
        "radius_limits": (None, None)
    }
}

## Vector Quantities

This defines the vector quantities to convert.

- **key**: tuple of strings. Names of the x, y, and z components as they should appear in VAPoR. Optional fourth name for the magnitude of the vector.
- **value**: tuple of quantity codes, for the x, y, and z components of the vector. Must have exactly three components, even if outputing the magnitude.

In [ ]:
vectors = {
    ("vx", "vy", "vz"): (1, 2, 3),
    ("wx", "wy", "wz", "wmag"): (301, 302, 303)
}

Editing the following code is not recommended. It mostly just unpacks those dictionaries and lets the `rayleigh_vapor` helper class do the rest.

In [ ]:
# DO NOT EDIT

from rayleigh_diagnostics import rayleigh_vapor , gen_3d_filelist

if run_dir[-1] != '/':
    run_dir = run_dir + '/'

if vapor_version == 2:
    vapor_file=vapor_file+'_v2'

var_codes = [v["code"] for v in scalars.values()]
var_names = [k for k in scalars]

remove_spherical_means = [v["rm_spherical_mean"] if "rm_spherical_mean" in v else False for v in scalars.values()]

rmins = [v["radius_limits"][0] if "radius_limits" in v else None for v in scalars.values()]
rmaxes = [v["radius_limits"][1] if "radius_limits" in v else None for v in scalars.values()]

scalar_files = gen_3d_filelist(var_codes,diter,first_iter,last_iter,directory=run_dir+data_dir)

vector_names = [key for key in vectors]
vector_codes = [value for value in vectors.values()]

vector_files = [
    gen_3d_filelist(codes, diter, first_iter, last_iter, directory=run_dir+data_dir) for codes in vector_codes
]

In [ ]:
# DO NOT EDIT

test = rayleigh_vapor(name=vapor_file,varnames=var_names,varfiles=scalar_files,vapor_bin=vbin,
                       nxyz=cube_size_N, grid_file=grid_file, rayleigh_root=rayleigh_root, force=True,
                       remove_spherical_means=remove_spherical_means, rmaxes=rmaxes, rmins=rmins, 
                       vapor_version=vapor_version,
                       vector_names=vector_names, vector_files=vector_files, tempdir=tempdir)
test.create_dataset(force=overwrite)

In [ ]:
test.populate_dataset()